In [ ]:
import general_robotics_toolbox as grt
import general_robotics_toolbox_invkin as igrt
from Arm_Lib import *
import numpy as np
import time
import math

#GLOBALS
NUM_JOINTS = 5
MIN_ANGLES = np.deg2rad(np.array([0,0,0,0,0]))
MAX_ANGLES = np.deg2rad(np.array([180,180,180,180,270]))
ZERO_CONFIG = np.deg2rad(np.array([90,90,90,90,135]))
in_2_m = 0.0254

In [ ]:
def eq_config(robot, config):
    # robot: Robot Object
    # config: list/tuple/array of angles whose length is equal 
    #           to that of the robot's number of joints
    # returns: an equivalent config that is within joint limits or None if not possible
    assert(len(config) == len(robot.joint_lower_limit))
    new_angles = [None]*len(config)
    for i in range(len(config)):
        new_angles[i] = config[i]
        theta = config[i]
        if theta < robot.joint_lower_limit[i] or theta > robot.joint_upper_limit[i]:
            up = theta + np.pi*2
            dw = theta - np.pi*2
            if up > robot.joint_lower_limit[i] and up < robot.joint_upper_limit[i]:
                new_angles[i] = up
            elif dw > robot.joint_lower_limit[i] and dw < robot.joint_upper_limit[i]:
                new_angles[i] = dw
            else:
                return None
    return np.array(new_angles)

In [ ]:
def digest_possible_configs(robot,configs):
    # robot: Robot Object
    # configs: (tuple of tuples) of possible configurations
    #           for the robot
    # returns: all configurations that are within
    #           the joint limits
    for i in range(len(configs)):
        assert(len(robot.joint_lower_limit) == len(configs[i]))
        assert(len(robot.joint_upper_limit) == len(configs[i]))

    new_configs = [None]*len(configs)
    num_invalid = 0
    for j in range(len(configs)):
        new_configs[i] = eq_config(robot,configs[i])
        if new_configs[i].any() == None:
            num_invalid += 1
    if (len(configs)-num_invalid) != 0:
        rv = [None]*(len(configs)-num_invalid)
    else:
        return None
    idx = 0
    for i in range(len(configs)):
        if new_configs[i] is not None:
            rv[idx] = new_configs[i]
            idx += 1
    return np.array(rv)
        

In [ ]:
def write_arm_deg(Arm,angles,d_time):
        """Writes angles to arm where angles
            where angles is a numpy array of 5 integers."""
        for i in range(1,6):
            Arm.Arm_serial_servo_write(i,angles[i-1],d_time)

In [ ]:
def getCurrentJointAngles():
    Arm = Arm_Lib.Arm_Device()
    angles = [0,0,0,0,0]
    for i in range(1,6):
        angles[i-1] = Arm.Arm_serial_servo_read(i)
    return angles

In [ ]:
def dofbot_robot():
    """REturns an approximate Robot instance for a DOFBOT robot"""
    x = np.array([1,0,0])
    y = np.array([0,1,0])
    z = np.array([0,0,1])
    a = np.array([0,0,0])
    
    H = np.array([z,y,y,y,z]).T
    P = np.array([4.5*z,a,3.5*z,3.5*z,3.0*z,4.5*z]).T*in_2_m
    JTs = [0,0,0,0,0]
    return grt.Robot(H,P,JTs,MIN_ANGLES,MAX_ANGLES)

In [ ]:
def generate_path(robot, desired_pose, q_current, iterations, tolerance=1e-4):
    """
        robot: Robot Object
        desired_pose: Transform Object; the desired pose of the robot -> gotten from fwdkin
        q_current: current joint angles ( in rad ) -> gotten from np.deg2rad(getCurrentJointAngles())
        iterations: positive integer -> Number of subpaths generated -> smaller means less accurate.
                    (aka 1/Lambda)
        tolerance: positive integer -> convergence tolerance ( read return below )
        returns: list of consecutive configs where the start config is the current
                    and the final config is as close to the desired_pose as possible (within tolerance)
    """
    path = np.array(np.array([q_current] + ([None]*(iterations-1))))
    scale_factor = 1/(iterations-1)
    initial_pose = grt.fwdkin(robot,q_current)
    poses = np.array([initial_pose] + ([None]*(iterations-1)))
    
    p_initial = initial_pose.p
    p_final = desired_pose.p
    r_initial = initial_pose.R
    r_final = desired_pose.R
    p_iteration = (p_final - p_initial)*scale_factor
    r_iteration = (r_final - r_initial)*scale_factor
    for scaled_lambda in range(1,iterations):
        previous_p = poses[scaled_lambda-1].p
        previous_r = poses[scaled_lambda-1].R
        this_pose = grt.Transform(p=previous_p+p_iteration,R=previous_r+r_iteration)
        # TODO instead of just putting calculated pose -> move thena
        #       check the current pose and adjust an error metric
        poses[scaled_lambda] = this_pose
        q_invkin = igrt.iterative_invkin(robot,this_pose,path[scaled_lambda-1])
        configs = digest_possible_configs(robot,q_invkin[1])
        # TODO Select min distance config from current config
        path[scaled_lambda] = configs[0]

        print("path[" +str(scaled_lambda) + "]:")
        print(path[scaled_lambda])

    return path

In [ ]:
def main():
    MEFA_BOT = dofbot_robot()
    TEST1 = [90,90,90,50,135]
    desired_pose = grt.fwdkin(MEFA_BOT,np.deg2rad(np.array(TEST1)))
    print("desired_pose:")
    print(desired_pose)
    angles = np.deg2rad(getCurrentJointAngles())
    qs = igrt.iterative_invkin(MEFA_BOT,desired_pose,angles,max_steps=250)
    print("desired angles: ")
    print(TEST1)
    print(np.rad2deg(qs[1]))
    oh_god_please = digest_possible_configs(MEFA_BOT,qs[1])
    if oh_god_please is not None:
        print(np.rad2deg(oh_god_please[0]))

    print("BEGIN PATH TEST")
    final_pose = grt.fwdkin(MEFA_BOT,np.deg2rad(np.array(TEST1)))
    new_path = generate_path(MEFA_BOT,final_pose,np.deg2rad(getCurrentJointAngles()),20)
    print(new_path)
    

In [51]:
if __name__ == "__main__":
    Arm = Arm_Lib.Arm_Device()
    write_arm_deg(Arm,np.rad2deg(ZERO_CONFIG),50)
    print("--STARTING--")
    time.sleep(.2)
    main()
    print("--END--")
    del Arm
    

--STARTING--
R = [[1. 0. 0.]
     [0. 1. 0.]
     [0. 0. 1.]]
p = [0.     0.     0.4826]


--END--
